In [1]:
import loompy
import numpy as np
import pandas as pd
from anndata import AnnData
import scanpy as sc

In [3]:
lc = loompy.connect("./raw/zeisel_2018/l5_all.loom")

In [4]:
X = lc.layers[""].sparse().T.tocsr() 
dict_obs = dict()
for k in lc.col_attrs.keys():
    if k == "Comment":
        continue
    dict_obs[k] = lc.col_attrs[k]
obs = pd.DataFrame(dict_obs).set_index("CellID")
var = pd.DataFrame(dict(lc.row_attrs)).set_index("Gene")

In [5]:
adata = AnnData(X, obs=obs, var=var)
adata.obs_names_make_unique()
adata = adata[:, ~adata.var.index.duplicated(keep=False)].copy()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [8]:
df_hom = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/gene_annotation/'
                     'mouse_human_homologs.txt', sep='\t')

In [12]:
adata = adata[:, adata.var.index.isin(df_hom["MOUSE_GENE_SYM"])]
adata.write_h5ad("processed/zeisel_2018.raw.h5ad")

/n/holystore01/LABS/price_lab/Users/khou/miniconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1207: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.obs` of view, copying.
... storing 'Age' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'AnalysisPool' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'AnalysisProject' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Bucket' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'CellConc' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Cell_Conc' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ChipID' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'Class' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'ClusterName' as categorical
Trying to set attribute `.ob

In [11]:
# adata_hippo = adata[adata.obs["Region"].isin(["Hippocampus"]), :]
# # filter homolog genes
# adata_hippo = adata_hippo[:, adata_hippo.var.index.isin(df_hom["MOUSE_GENE_SYM"])]
# # save 
# adata_hippo.write_h5ad("raw.h5ad")

In [2]:
adata = sc.read_h5ad("processed/zeisel_2018.raw.h5ad")

In [3]:
import scanpy as sc
sc.pp.filter_cells(adata, min_genes=0)
df_cov = adata.obs[["n_genes"]]
df_cov["const"] = 1.
df_cov.to_csv("processed/zeisel_2018.cov.tsv", sep='\t')

/n/holystore01/LABS/price_lab/Users/khou/miniconda3/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
<ipython-input-3-e6beafb192f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cov["const"] = 1.
